<a href="https://colab.research.google.com/github/maggiemcc02/HessianMonitorWork/blob/main/Hessian_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook I learn how to compute the $|H(u)|$ in firedrake using chosen $u(x(\xi))$ (so that we know what $|H(u)|$ should be).

In [ ]:
# install firedrake

# hide output
%%capture

try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [ ]:
# import firedrake tools
%matplotlib notebook
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d

import scipy as sci

# Test 1

In [ ]:
def known_solution(x):


  u_exact = x[0]**2 + x[1]**2 + x[0]*x[1]

  return u_exact

In [ ]:
N = 24
mesh = UnitSquareMesh(N, N)
xi, eta = SpatialCoordinate(mesh)
V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 2)

x = Function(V).interpolate(as_vector([xi, eta]))

In [ ]:
known = Function(W).interpolate(known_solution(x))

J = grad(x)
J_inv = inv(J)
J_inv_T = J_inv.T


gradu = dot( J_inv_T, grad(known) )

ux = gradu[0]
uy = gradu[1]


grad2ux = dot( J_inv_T, grad(ux) )
grad2uy = dot( J_inv_T, grad(uy) )

u_xx = grad2ux[0]
u_yy = grad2uy[1]
u_yx = grad2uy[0]
u_xy = grad2ux[1]


# u_xx = Function(W).interpolate(grad2ux[0])
# u_yy = Function(W).interpolate(grad2uy[1])
# u_yx = Function(W).interpolate(grad2uy[0])
# u_xy = Function(W).interpolate(grad2ux[1])

# print(u_xx.dat.data)
# print()
# print(u_yy.dat.data)
# print()
# print(u_xy.dat.data)
# print()
# print(u_yx.dat.data)

In [ ]:
# construct the Hessian

Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])


# find eigenvalues

aH = u_xx
bH = u_yy
cH = u_xy

L1 = ((aH + bH) + sqrt((aH - bH)**2 + 4*cH**2))/2
L2 = ((aH + bH) - sqrt((aH - bH)**2 + 4*cH**2))/2

# Find eigenvectors

sc1 = 1/sqrt(cH**2 + (L1 - aH)**2 )
sc2 = 1/sqrt(cH**2 + (L2 - aH)**2 )
v1 = sc1 * as_vector([cH, L1 - aH])
v2 = sc2 * as_vector([cH, L2 - aH])

# construct Q matrices

Q = as_matrix([[v1[0], v2[0]], [v1[1], v2[1]]])
Q_inv = inv(Q)

# construct lambda matrix

abs_L = as_matrix([[abs(L1), 0], [0, abs(L2)]])


# print the results

print('The second partial derivatives')
print('uxx = ', Function(W).interpolate(Hu[0,0]).dat.data) #uxx
print('uxy = ', Function(W).interpolate(Hu[0,1]).dat.data) #uxy
print('uyx = ', Function(W).interpolate(Hu[1,0]).dat.data) #uyx
print('uyy = ', Function(W).interpolate(Hu[1,1]).dat.data) #uyy
print()
print()


# construct the Hessian

Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])


# find eigenvalues

aH = u_xx
bH = u_yy
cH = u_xy

L1 = ((aH + bH) + sqrt((aH - bH)**2 + 4*cH**2))/2
L2 = ((aH + bH) - sqrt((aH - bH)**2 + 4*cH**2))/2


print("The eigenvalues")
print(Function(W).interpolate(L1).dat.data)
print(Function(W).interpolate(L2).dat.data)
print()
print()

# Find eigenvectors

sc1 = 1/sqrt(cH**2 + (L1 - aH)**2 )
sc2 = 1/sqrt(cH**2 + (L2 - aH)**2 )
v1 = sc1 * as_vector([cH, L1 - aH])
v2 = sc2 * as_vector([cH, L2 - aH])

print("The eigenvectors")
print(Function(V).interpolate(v1).dat.data)
print()
print(Function(V).interpolate(v2).dat.data)
print()
print()

# construct Q matrices

Q = as_matrix([[v1[0], v2[0]], [v1[1], v2[1]]])
Q_inv = inv(Q)

print('The Q matrix')
print(Function(W).interpolate(Q[0, 0]).dat.data)
print(Function(W).interpolate(Q[0, 1]).dat.data)
print(Function(W).interpolate(Q[1, 0]).dat.data)
print(Function(W).interpolate(Q[1, 1]).dat.data)
print()
print()

print('The Inverse of Q')
print(Function(W).interpolate(Q_inv[0, 0]).dat.data)
print(Function(W).interpolate(Q_inv[0, 1]).dat.data)
print(Function(W).interpolate(Q_inv[1, 0]).dat.data)
print(Function(W).interpolate(Q_inv[1, 1]).dat.data)
print()
print()


# construct lambda matrix

abs_L = as_matrix([[abs(L1), 0], [0, abs(L2)]])

print('Absolute Value of Lambda Matrix')
print(Function(W).interpolate(abs_L[0, 0]).dat.data)
print(Function(W).interpolate(abs_L[1, 1]).dat.data)
print()
print()



# Construct the absolute value of H(u)

abs_Hu = dot(Q, dot(abs_L, Q_inv))


print('Components of |H(u)|')
print(Function(W).interpolate(abs_Hu[0,0]).dat.data) #uxx
print( Function(W).interpolate(abs_Hu[0,1]).dat.data) #uxy
print(Function(W).interpolate(abs_Hu[1,0]).dat.data) #uyx
print(Function(W).interpolate(abs_Hu[1,1]).dat.data) #uyy
print()
print()


# Now construct I + |H(u)|

B = Identity(2) + abs_Hu

print('Components of B = I + |H(u)|')
print(Function(W).interpolate(B[0,0]).dat.data) #uxx
print( Function(W).interpolate(B[0,1]).dat.data) #uxy
print(Function(W).interpolate(B[1,0]).dat.data) #uyx
print(Function(W).interpolate(B[1,1]).dat.data) #uyy
print()
print()

The second partial derivatives
uxx =  [2. 2. 2. ... 2. 2. 2.]
uxy =  [1. 1. 1. ... 1. 1. 1.]
uyx =  [1. 1. 1. ... 1. 1. 1.]
uyy =  [2. 2. 2. ... 2. 2. 2.]


The eigenvalues
[3. 3. 3. ... 3. 3. 3.]
[1. 1. 1. ... 1. 1. 1.]


The eigenvectors
[[0.70710678 0.70710678]
 [0.70710678 0.70710678]
 [0.70710678 0.70710678]
 ...
 [0.70710678 0.70710678]
 [0.70710678 0.70710678]
 [0.70710678 0.70710678]]

[[ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 ...
 [ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]


The Q matrix
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[-0.70710678 -0.70710678 -0.70710678 ... -0.70710678 -0.70710678
 -0.70710678]


The Inverse of Q
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[0.70710678 0.70710678 0.70710678 

In [ ]:
# now with scipy


Hu_prod = dot( Hu.T, Hu)

Hu00 = Function(W).interpolate(Hu_prod[0,0])
Hu01 = Function(W).interpolate(Hu_prod[0,1])
Hu10 = Function(W).interpolate(Hu_prod[1,0])
Hu11 = Function(W).interpolate(Hu_prod[1,1])


# place to store each sqrt matrix
n_comp =2401
# Initialize an array to store the results
absHu_sci = np.zeros((2, 2, n_comp))
# Compute the square root of each 2x2 matrix
for i in range(n_comp):
  Hu_matrix = np.array([[Hu00.dat.data[i], Hu01.dat.data[i]], [Hu10.dat.data[i], Hu11.dat.data[i]]])
  absHu_sci[:, :, i] = sci.linalg.sqrtm(Hu_matrix)

# print componenets

print("Computed components of |H(u)| as Numpy Array")
print(absHu_sci[0, 0, :])
print(absHu_sci[0, 1, :])
print(absHu_sci[1, 0, :])
print(absHu_sci[1, 1, :])
print()
print()

# make this a firedrake array


Hu00.dat.data[:] = absHu_sci[0, 0, :]
Hu01.dat.data[:] = absHu_sci[0, 1, :]
Hu10.dat.data[:] = absHu_sci[1, 0, :]
Hu11.dat.data[:] = absHu_sci[1, 1, :]

final_abs_Hu = as_matrix([[Hu00, Hu01], [Hu10, Hu11]])

print('Components of |H(u)| as a Firedrake Function')
print(Function(W).interpolate(final_abs_Hu[0,0]).dat.data) #uxx
print( Function(W).interpolate(final_abs_Hu[0,1]).dat.data) #uxy
print(Function(W).interpolate(final_abs_Hu[1,0]).dat.data) #uyx
print(Function(W).interpolate(final_abs_Hu[1,1]).dat.data) #uyy
print()
print()

Computed components of |H(u)| as Numpy Array
[2. 2. 2. ... 2. 2. 2.]
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[2. 2. 2. ... 2. 2. 2.]


Components of |H(u)| as a Firedrake Function
[2. 2. 2. ... 2. 2. 2.]
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[2. 2. 2. ... 2. 2. 2.]




# Test 1.5

In [ ]:
def known_solution(x):


  u_exact = x[0]**2 / 2 + x[1]**2 / 2 + 2 * x[0]*x[1]

  return u_exact

In [ ]:
N = 24
mesh = UnitSquareMesh(N, N)
xi, eta = SpatialCoordinate(mesh)
V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 2)

x = Function(V).interpolate(as_vector([xi, eta]))

In [ ]:
known = Function(W).interpolate(known_solution(x))

J = grad(x)
J_inv = inv(J)
J_inv_T = J_inv.T


gradu = dot( J_inv_T, grad(known) )

ux = gradu[0]
uy = gradu[1]


grad2ux = dot( J_inv_T, grad(ux) )
grad2uy = dot( J_inv_T, grad(uy) )

u_xx = grad2ux[0]
u_yy = grad2uy[1]
u_yx = grad2uy[0]
u_xy = grad2ux[1]


# u_xx = Function(W).interpolate(grad2ux[0])
# u_yy = Function(W).interpolate(grad2uy[1])
# u_yx = Function(W).interpolate(grad2uy[0])
# u_xy = Function(W).interpolate(grad2ux[1])

# print(u_xx.dat.data)
# print()
# print(u_yy.dat.data)
# print()
# print(u_xy.dat.data)
# print()
# print(u_yx.dat.data)

In [ ]:
# construct the Hessian

Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])


# find eigenvalues

aH = u_xx
bH = u_yy
cH = u_xy

L1 = ((aH + bH) + sqrt((aH - bH)**2 + 4*cH**2))/2
L2 = ((aH + bH) - sqrt((aH - bH)**2 + 4*cH**2))/2

# Find eigenvectors

sc1 = 1/sqrt(cH**2 + (L1 - aH)**2 )
sc2 = 1/sqrt(cH**2 + (L2 - aH)**2 )
v1 = sc1 * as_vector([cH, L1 - aH])
v2 = sc2 * as_vector([cH, L2 - aH])

# construct Q matrices

Q = as_matrix([[v1[0], v2[0]], [v1[1], v2[1]]])
Q_inv = inv(Q)

# construct lambda matrix

abs_L = as_matrix([[abs(L1), 0], [0, abs(L2)]])


# print the results

print('The second partial derivatives')
print('uxx = ', Function(W).interpolate(Hu[0,0]).dat.data) #uxx
print('uxy = ', Function(W).interpolate(Hu[0,1]).dat.data) #uxy
print('uyx = ', Function(W).interpolate(Hu[1,0]).dat.data) #uyx
print('uyy = ', Function(W).interpolate(Hu[1,1]).dat.data) #uyy
print()
print()


# construct the Hessian

Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])


# find eigenvalues

aH = u_xx
bH = u_yy
cH = u_xy

L1 = ((aH + bH) + sqrt((aH - bH)**2 + 4*cH**2))/2
L2 = ((aH + bH) - sqrt((aH - bH)**2 + 4*cH**2))/2


print("The eigenvalues")
print(Function(W).interpolate(L1).dat.data)
print(Function(W).interpolate(L2).dat.data)
print()
print()

# Find eigenvectors

sc1 = 1/sqrt(cH**2 + (L1 - aH)**2 )
sc2 = 1/sqrt(cH**2 + (L2 - aH)**2 )
v1 = sc1 * as_vector([cH, L1 - aH])
v2 = sc2 * as_vector([cH, L2 - aH])

print("The eigenvectors")
print(Function(V).interpolate(v1).dat.data)
print()
print(Function(V).interpolate(v2).dat.data)
print()
print()

# construct Q matrices

Q = as_matrix([[v1[0], v2[0]], [v1[1], v2[1]]])
Q_inv = inv(Q)

print('The Q matrix')
print(Function(W).interpolate(Q[0, 0]).dat.data)
print(Function(W).interpolate(Q[0, 1]).dat.data)
print(Function(W).interpolate(Q[1, 0]).dat.data)
print(Function(W).interpolate(Q[1, 1]).dat.data)
print()
print()

print('The Inverse of Q')
print(Function(W).interpolate(Q_inv[0, 0]).dat.data)
print(Function(W).interpolate(Q_inv[0, 1]).dat.data)
print(Function(W).interpolate(Q_inv[1, 0]).dat.data)
print(Function(W).interpolate(Q_inv[1, 1]).dat.data)
print()
print()


# construct lambda matrix

abs_L = as_matrix([[abs(L1), 0], [0, abs(L2)]])

print('Absolute Value of Lambda Matrix')
print(Function(W).interpolate(abs_L[0, 0]).dat.data)
print(Function(W).interpolate(abs_L[1, 1]).dat.data)
print()
print()



# Construct the absolute value of H(u)

abs_Hu = dot(Q, dot(abs_L, Q_inv))


print('Components of |H(u)|')
print(Function(W).interpolate(abs_Hu[0,0]).dat.data) #uxx
print( Function(W).interpolate(abs_Hu[0,1]).dat.data) #uxy
print(Function(W).interpolate(abs_Hu[1,0]).dat.data) #uyx
print(Function(W).interpolate(abs_Hu[1,1]).dat.data) #uyy
print()
print()


# construct the desired B matrix

B = Identity(2) + abs_Hu

print('Components of B = I + |H(u)|')
print(Function(W).interpolate(B[0,0]).dat.data) #uxx
print( Function(W).interpolate(B[0,1]).dat.data) #uxy
print(Function(W).interpolate(B[1,0]).dat.data) #uyx
print(Function(W).interpolate(B[1,1]).dat.data) #uyy
print()
print()

The second partial derivatives
uxx =  [1. 1. 1. ... 1. 1. 1.]
uxy =  [2. 2. 2. ... 2. 2. 2.]
uyx =  [2. 2. 2. ... 2. 2. 2.]
uyy =  [1. 1. 1. ... 1. 1. 1.]


The eigenvalues
[3. 3. 3. ... 3. 3. 3.]
[-1. -1. -1. ... -1. -1. -1.]


The eigenvectors
[[0.70710678 0.70710678]
 [0.70710678 0.70710678]
 [0.70710678 0.70710678]
 ...
 [0.70710678 0.70710678]
 [0.70710678 0.70710678]
 [0.70710678 0.70710678]]

[[ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 ...
 [ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]


The Q matrix
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[-0.70710678 -0.70710678 -0.70710678 ... -0.70710678 -0.70710678
 -0.70710678]


The Inverse of Q
[0.70710678 0.70710678 0.70710678 ... 0.70710678 0.70710678 0.70710678]
[0.70710678 0.70710678 0.707

In [ ]:
# now with scipy


Hu_prod = dot( Hu.T, Hu)

Hu00 = Function(W).interpolate(Hu_prod[0,0])
Hu01 = Function(W).interpolate(Hu_prod[0,1])
Hu10 = Function(W).interpolate(Hu_prod[1,0])
Hu11 = Function(W).interpolate(Hu_prod[1,1])


# place to store each sqrt matrix
n_comp =2401
# Initialize an array to store the results
absHu_sci = np.zeros((2, 2, n_comp))
# Compute the square root of each 2x2 matrix
for i in range(n_comp):
  Hu_matrix = np.array([[Hu00.dat.data[i], Hu01.dat.data[i]], [Hu10.dat.data[i], Hu11.dat.data[i]]])
  absHu_sci[:, :, i] = sci.linalg.sqrtm(Hu_matrix)

# print componenets

print("Computed components of |H(u)| as Numpy Array")
print(absHu_sci[0, 0, :])
print(absHu_sci[0, 1, :])
print(absHu_sci[1, 0, :])
print(absHu_sci[1, 1, :])
print()
print()

# make this a firedrake array


Hu00.dat.data[:] = absHu_sci[0, 0, :]
Hu01.dat.data[:] = absHu_sci[0, 1, :]
Hu10.dat.data[:] = absHu_sci[1, 0, :]
Hu11.dat.data[:] = absHu_sci[1, 1, :]

final_abs_Hu = as_matrix([[Hu00, Hu01], [Hu10, Hu11]])

print('Components of |H(u)| as a Firedrake Function')
print(Function(W).interpolate(final_abs_Hu[0,0]).dat.data) #uxx
print( Function(W).interpolate(final_abs_Hu[0,1]).dat.data) #uxy
print(Function(W).interpolate(final_abs_Hu[1,0]).dat.data) #uyx
print(Function(W).interpolate(final_abs_Hu[1,1]).dat.data) #uyy
print()
print()

Computed components of |H(u)| as Numpy Array
[2. 2. 2. ... 2. 2. 2.]
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[2. 2. 2. ... 2. 2. 2.]


Components of |H(u)| as a Firedrake Function
[2. 2. 2. ... 2. 2. 2.]
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[2. 2. 2. ... 2. 2. 2.]




# Test 2

# Test 2

In [ ]:
def known_solution(x):


  u_exact = sin(x[0])*cos(x[1])

  return u_exact

In [ ]:
N = 24
mesh = UnitSquareMesh(N, N)
xi, eta = SpatialCoordinate(mesh)
V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 2)

x = Function(V).interpolate(as_vector([xi, eta]))

In [ ]:
known = known_solution(x)

J = grad(x)
J_inv = inv(J)
J_inv_T = J_inv.T


gradu = dot( J_inv_T, grad(known) )

ux = gradu[0]
uy = gradu[1]


grad2ux = dot( J_inv_T, grad(ux) )
grad2uy = dot( J_inv_T, grad(uy) )


u_xx = Function(W).interpolate(grad2ux[0])
u_yy = Function(W).interpolate(grad2uy[1])
u_yx = Function(W).interpolate(grad2uy[0])
u_xy = Function(W).interpolate(grad2ux[1])

# known results

uxx_k = Function(W).interpolate(-sin(x[0])*cos(x[1]))
uxy_k = Function(W).interpolate(-cos(x[0])*sin(x[1]) )

print(uxx_k.dat.data - u_xx.dat.data)
print()
print(uxx_k.dat.data - u_yy.dat.data)
print()
print(uxy_k.dat.data - u_xy.dat.data)
print()
print(uxy_k.dat.data - u_yx.dat.data)

# Test 3

In [ ]:
def known_solution(x):


  u_exact = x[0]**3 + x[1]**2 + x[0]*x[1]

  return u_exact

In [ ]:
N = 24
mesh = UnitSquareMesh(N, N)
xi, eta = SpatialCoordinate(mesh)
V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 2)

x = Function(V).interpolate(as_vector([xi, eta]))

In [ ]:
known = known_solution(x)

J = grad(x)
J_inv = inv(J)
J_inv_T = J_inv.T


gradu = dot( J_inv_T, grad(known) )

ux = gradu[0]
uy = gradu[1]


grad2ux = dot( J_inv_T, grad(ux) )
grad2uy = dot( J_inv_T, grad(uy) )


u_xx = Function(W).interpolate(grad2ux[0])
u_yy = Function(W).interpolate(grad2uy[1])
u_yx = Function(W).interpolate(grad2uy[0])
u_xy = Function(W).interpolate(grad2ux[1])

# known results

uxx_k = Function(W).interpolate(6*x[0])


print(u_xx.dat.data)
print()
print(uxx_k.dat.data)
print()
print(u_xx.dat.data - uxx_k.dat.data)
print()
print(u_yy.dat.data)
print()
print(u_xy.dat.data)
print()
print(u_yx.dat.data)

# Test 4

In [ ]:
def known_solution(x):


  u_exact = exp(x[0]) * cos(x[1])

  return u_exact

In [ ]:
N = 24
mesh = UnitSquareMesh(N, N)
xi, eta = SpatialCoordinate(mesh)
V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 2)

x = Function(V).interpolate(as_vector([xi, eta]))

In [ ]:
known = known_solution(x)

J = grad(x)
J_inv = inv(J)
J_inv_T = J_inv.T


gradu = dot( J_inv_T, grad(known) )

ux = gradu[0]
uy = gradu[1]


grad2ux = dot( J_inv_T, grad(ux) )
grad2uy = dot( J_inv_T, grad(uy) )


u_xx = Function(W).interpolate(grad2ux[0])
u_yy = Function(W).interpolate(grad2uy[1])
u_yx = Function(W).interpolate(grad2uy[0])
u_xy = Function(W).interpolate(grad2ux[1])

# known results

uxx_k = Function(W).interpolate(exp(x[0])*cos(x[1]))
uxy_k = Function(W).interpolate(-exp(x[0])*sin(x[1]))
uyy_k = Function(W).interpolate(-exp(x[0])*cos(x[1]))

print(uxx_k.dat.data - u_xx.dat.data)
print()
print(uyy_k.dat.data - u_yy.dat.data)
print()
print(uxy_k.dat.data - u_xy.dat.data)
print()
print(uxy_k.dat.data - u_yx.dat.data)